In [ ]:
class Board:
    def __init__(self):
        self.state = [[0,0,0] for _ in range(3)]
    def set_board_from_file(self,filename):
        pass

class Solver:
    def __init__(self,board: Board):
        self.board = board
    def solve_btc(self):
        pass

    def solve_fcc(self):
        pass

    def solve(self,algorithm: str):
        """

        :param algorithm: Takes an argument for solving method, accepted values are:
            btc: backtracking
            fcc: forwardchecking
            ...
        :return:
        """
        if algorithm == "btc":
            self.solve_btc()